# Copying Apriori Algo 

In [90]:
from collections import defaultdict  # if key unregistered instead of returning KeyError return 0.
# blank for imports
from itertools import chain, combinations

In [91]:
def subsets(arr):
    """ Returns non-empty subsets of arr"""
    return chain(
        *[combinations(arr, i + 1) for i, a in enumerate(arr)])  # Σ{k=1->n}(C(n,k)) = Σ{k=1->n} ( n! / (k! * (n-k)! )


def getItemSetTransactionList(data_iterator):
    """ Transaction list is the list of records of the data (csv)"""
    transactionList = list()
    itemSet = set()
    for record in data_iterator:
        transaction = frozenset(record)
        transactionList.append(transaction)
        for item in transaction:
            itemSet.add(frozenset([item]))  # Generate 1-itemSets
    return itemSet, transactionList


def returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet):
    """ Calculates the support for items in the itemSet and returns a subset of the
        itemSet each of whose elements satisfies the minimum support"""
    _itemSet = set()
    localSet = defaultdict(int)

    for item in itemSet:
        for transaction in transactionList:
            if item.issubset(transaction):  # we use is subset cause item could be a set
                freqSet[item] += 1
                localSet[item] += 1

    for item, count in localSet.items():
        support = float(count) / len(transactionList)

        if support >= minSupport:
            _itemSet.add(item)

    return _itemSet


def joinSet(itemSet: set, length: int) -> set:
    """ Join a set with itself and returns an n-element itemSet"""
    return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])

In [92]:
"""# =========================================================================================== #
# data_iter = frozenset(['beer', 'rice', 'beer'])
# for i in subsets(data_iter):
#     print(i)"""
import pandas as pd
from pprint import pprint

# data = pd.read_csv("git_apriori/Apriori-python3/tesco.csv", header=None)

data = pd.read_csv('vac_data/vacadays_simple.csv')

"""print("Header:", list(data.columns))
print("First row:");print( data.iloc[0])
print("First Row without headers")
print("\t".join(map(str, data.iloc[0])))
print("="*2**6)"""
data = data[['Opening Hours', 'Category']]
data = data.dropna()  # we want a pair so nan means there is no pair so useless
print(data)

                                         Opening Hours          Category
0    Tuesday: [11 AM-11 PM], Wednesday: [11 AM-11 P...   Food & Beverage
1    Tuesday: [8 AM-8 PM], Wednesday: [8 AM-8 PM], ...   Food & Beverage
2    Tuesday: [10 AM-2 AM], Wednesday: [10 AM-2 AM]...  Bars & Nightlife
3    Tuesday: [8:30 AM-8:30 PM], Wednesday: [8:30 A...   Food & Beverage
4    Tuesday: [10 AM-6 PM], Wednesday: [10 AM-6 PM]...   Food & Beverage
..                                                 ...               ...
239  Friday: [8 AM-1 AM], Saturday: [8 AM-1 AM], Su...   Food & Beverage
240  Friday: [12-11 PM], Saturday: [12-11 PM], Sund...   Food & Beverage
241  Friday: [12 PM-12 AM], Saturday: [12 PM-12 AM]...   Food & Beverage
242  Friday: [9 AM-12 AM], Saturday: [9 AM-12 AM], ...   Food & Beverage
243  Friday: [12 PM-12 AM], Saturday: [12 PM-12 AM]...   Food & Beverage

[225 rows x 2 columns]


In [114]:
"""row = data.iloc[0].values[0]
row = row.replace('\u202f', '')
row"""
timetable_data = []
for i, row in data.iterrows():
    cleaned_str = (row.values[0].replace('\u202f', '')
                   .replace('[', '').replace(']', ''))
    slots = cleaned_str.split(', ')
    timetable_data.append(slots)

stores = {}
import dateutil.parser


def pair_to_24(open_hours: str) -> str:
    if open_hours == 'Closed':
        return 'Closed'
    elif open_hours == 'Open 24 hours':
        return 'Open 24 hours'
    else:
        res = ''
        try:
            am_to_24 = lambda am_time: str(dateutil.parser.parse(am_time).hour)
            start, end = open_hours.split('-')
            end = am_to_24(end)
            start = am_to_24(start)
            res += start + '-' + end
        except Exception as e:
            return 'wrong_input' + open_hours
    return res


def get_times_of_day(time_range):
    """Gets the times of day for a given time range.
  
    Args:
      time_range: The time range in the format "start_time-end_time".
  
    Returns:
      A list of the times of day for the given time range.
    """

    start_time, end_time = time_range.split("-")

    start_hour = int(start_time[0:2])
    end_hour = int(end_time[0:2])

    times_of_day = []

    if start_hour <= 6 and end_hour <= 6:
        times_of_day.append("night")
    elif start_hour <= 12 and end_hour <= 12:
        times_of_day.append("morning")
    else:
        times_of_day.append("evening")

    if len(times_of_day) > 1:
        if "evening" in times_of_day and "morning" in times_of_day:
            times_of_day = ["both evening and morning"]

    return times_of_day

for store, store_timetable in enumerate(timetable_data):
    timetable = {}
    for slot in store_timetable:
        day, open_hours = slot.split(': ')

        timetable[day] = get_times_of_day(pair_to_24(open_hours))
    stores[store] = timetable


pprint(stores)

σαε φαιλ
ψες φαιλ
φιλοσοφια φαιλ
λειτουεγικα 9
βασεις πασσ
τν πασσ
εργαστηριο 7
αρχιτεκτονική πασσ
εφαρμο 7


SyntaxError: invalid syntax (1508454646.py, line 74)

10-16


In [30]:
no_nan_data = []
for i, row in data.iterrows():
    record = [item for item in row.values if not pd.isna(item)]
    no_nan_data.append(record)
    print(i, record)

itemSet, tranList = getItemSetTransactionList(no_nan_data)
# pprint(itemSet)
# pprint(tranList)

AttributeError: 'list' object has no attribute 'replace'

In [5]:
def runApriori(data_iter, minSupport, minConfidence):
    """
    run the apriori algorithm. data_iter is a record iterator
    :return: 
        - items (tuple, support)
        - rules ((pre-tuple, post-tuple), confidence)
    """
    itemSet, transactionList = getItemSetTransactionList(data_iter)

    freqSet = defaultdict(int)
    largeSet = dict()
    # Global dictionary which stores (key = n-itemSets, value=support) which satisfy minSupport

    assocRules = dict()  # Dict which stores Association Rules

    # has single elements no sets that achieve minimum support
    oneCSet = returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet)

    currentLSet = oneCSet
    k = 2
    while currentLSet != set([]):
        largeSet[k - 1] = currentLSet
        currentLSet = joinSet(currentLSet, k)
        currentCSet = returnItemsWithMinSupport(currentLSet, transactionList, minSupport, freqSet)
        currentLSet = currentCSet
        k += 1

    # Local function which Returns the support of an item { P(X) = n(X) / n(D) }
    getSupport = lambda item: float(freqSet[item]) / len(transactionList)

    toRetItems = []
    for key, value in largeSet.items():
        # The reason we get item in tuple is that again item can be a set of items
        toRetItems.extend([(tuple(item), getSupport(item)) for item in value])

    # NOW THE SECOND PART: THE RULES

    toRetRules = []
    for key, value in list(largeSet.items())[1:]:  # why not the first one?
        for item in value:
            _subsets = map(frozenset, [x for x in subsets(item)])
            for element in _subsets:
                remain = item.difference(element)  # create and check all possible antecedents 
                if len(remain) > 0:
                    # confidence = the proportion of the number of instances to which the rule applies
                    # The probability that a consequence will occur if the antecedent occurs: Conf(X->Y) = P(Y/X) = P(X U Y) / P(X)
                    confidence = getSupport(item) / getSupport(element)
                    if confidence >= minConfidence:
                        toRetRules.append(((tuple(element), tuple(remain)), confidence))

    return toRetItems, toRetRules


In [9]:
items, rules = runApriori(no_nan_data, 0.1, 1)
pprint(items)
print()
print()
pprint(rules)

[(('Bar',), 0.13247863247863248),
 (('Museums & Sightseeing',), 0.13247863247863248),
 (('Bars & Nightlife',), 0.2264957264957265),
 (('Restaurant',), 0.20512820512820512),
 (('Food & Beverage',), 0.5598290598290598),
 (('All Day',), 0.15384615384615385),
 (('Food & Beverage', 'Restaurant'), 0.20085470085470086),
 (('Bar', 'Bars & Nightlife'), 0.13247863247863248)]


[((('Bar',), ('Bars & Nightlife',)), 1.0)]
